# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. **(A to już znamy!!!)**

## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [1]:
import numpy as np
import random

# check prime naive
def check_prime_naive(n):
    if n == 2:
        return True
    if n < 2 or n % 2 == 0:
        return False
    for i in range(3, int(np.sqrt(n)+1), 2):
        if n % i == 0:
            return False
    return True

SIZE = 10_001
IS_PRIME_10K = [check_prime_naive(n) for n in range(SIZE)]
PRIMES_10K = [n for n in range(SIZE) if IS_PRIME_10K[n]]

In [2]:
# miller rabin test
def miller_rabin_test(n, iters=1):
    if n < 2: return False
    if n == 2 or n == 3: return True
    s = 0
    while (n-1)%(2**s) == 0:
        s += 1
    s -= 1
    d = (n-1)//(2**s)

    for _ in range(iters):
        a = random.randint(2, n-2)
        x = pow(a, d, mod=n)
        if x % n == 1:
            continue
        flag = True
        for _ in range(s):
            if x % n == n-1:
                flag = False
                break
            x = pow(x, 2, mod=n)
        if flag:
            return False
    return True

# generate big prime
def generate_prime(n):
    while True:
        # random number from 1 to 0b1111111...1 (n times)
        num = random.randint(1, (1 << n)-1)
        # set first and last bit to 1
        num = num | ((1 << n-1) + 1)
        # check divisibility for primes < 2000
        flag = False
        for p in PRIMES_10K:
            if num % p == 0:
                flag = True
                break
        if flag:
            continue
        # if miller-rabin passed - assume number is prime
        if miller_rabin_test(num, iters=10):
            break
    return num

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$, $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$ **(potrzebujemy rozszerzonego algorytmu Euklidesa!!!)**
5. Kluczem publicznym jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [3]:
def extended_euclid(a, b):
    r0, r1 = a, b
    s0, s1 = 1, 0
    t0, t1 = 0, 1
    while r1 != 0:
        q = int(r0/r1)
        r0, r1 = r1, r0 - q*r1
        s0, s1 = s1, s0 - q*s1
        t0, t1 = t1, t0 - q*t1
    return r0, s0, t0

def modulo_inverse(a, n):
    d, s, _ = extended_euclid(a, n)
    if d != 1:
        print("d != 1 - no inverse modulo")
        return None
    return s if s >= 0 else s+n

def generate_key(key_size):
    p = generate_prime(key_size)
    q = generate_prime(key_size)
    n = p*q
    totient = (p-1)*(q-1)

    r = -1
    while r != 1:
        e = random.randint(2, totient-1)
        r, _, _ = extended_euclid(e, totient)
    d = modulo_inverse(e, totient)

    public_key = (n, e)
    private_key = (n, d)
    return public_key, private_key

def make_key_files(key_size):
    public, private = generate_key(key_size)
    f = open("public_rsa", "w")
    f.write(f"{public}")
    f.close()
    f = open("private_rsa", "w")
    f.write(f"{private}")
    f.close()
    print(f"Public: {public}")
    print(f"Private: {private}")

def read_key_file(key_type):
    f = open(f"{key_type}_rsa", "r")
    key = eval(f.read())
    f.close()
    return key

print("Generated public and private keys:")
make_key_files(1024)

Generated public and private keys:
Public: (18327095718526884095455831710578475197693443797307121684740673897365133202849089679036226291867020138369278241850568489530878077678317148955075196024331984821516355255339983793982218224650034512256225549454010424310650061048527190171770329623559269034999873425964310503601919389854994030244617183548515330629435985286082591925416752452762967296669400038757956802539597396169083701347704554676145568789281607710319324694108413787128271179632411927199184898664947071462031066895508609913941028842892122241940055376612020152171929714843795322154015263486052189608892229325840350667428762275482431907393255040593459899923, 8045255897750817193073963702834064230799497165280292894409750545029449467826884881140765438513110443168223456404078717266499214993637147979890859124876260735045768222801309471569255549333167315535903189233705767751173469425858793715057771813254872253430846560027904008672555674185665969199126082481582350546945042263402856121135839928551

## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [4]:
def encrypt(message, modulus, exp):
    message_encrypted = list()
    for c in message:
        message_encrypted.append(pow(ord(c), exp, mod=modulus))
    return message_encrypted

### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [5]:
def decrypt(message_encrypted, modulus, exp):
    message_ascii = list()
    for c in message_encrypted:
        message_ascii.append(chr(pow(c, exp, mod=modulus)))
    return ('').join(message_ascii)

### Test

In [16]:
public = read_key_file("public")
private = read_key_file("private")

message = "the quick brown fox jumps over the lazy dog"
message_encrypted = encrypt(message, public[0], public[1])
message_decrypted = decrypt(message_encrypted, private[0], private[1])

print(f"Message: {message}")
print(f"Message encrypted:\n{",\n".join(map(str, message_encrypted))}")
print(f"Message decrypted: {message_decrypted}")

Message: the quick brown fox jumps over the lazy dog
Message encrypted:
...
Message decrypted: the quick brown fox jumps over the lazy dog


## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy) - zweryfikuj jak na wydajnosć wpływa zastosowanie różnych sposobów potęgowania dostępnych w Python i jego bibliotekach.  
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


### Algorytm szybkiego potęgowania 
1. Zwykłe potęgowanie $n^{exp}$: $exp$ mnożeń 
2. Algorytm szybkiego potęgowania: część mnożeń zastępujemy podnoszeniem do kwadratu (_squaring_).
    __Skąd mamy wiedzieć kiedy mnożyć, a kiedy potęgować?__

In [17]:
def fastModularExponentation(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m    